In [1]:
#%matplotlib inline
import numpy as np
from scipy.misc import imread
import pickle
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
import tensorflow as tf
from keras.preprocessing import image

from keras.backend.tensorflow_backend import set_session
from ssd import SSD300
from keras.applications.imagenet_utils import preprocess_input
from ssd_utils import BBoxUtility
import matplotlib.pyplot as plt
from SSD_tester import calc_detection_prec_rec, calc_detection_ap, calc_detection_ap_recall
from object_detection.utils.object_detection_evaluation import ObjectDetectionEvaluator
from object_detection.core import standard_fields
import time
import sys
a = time.time()

config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list='1',
        allow_growth=True 
    )
)
sess = sess = tf.Session(config=config)



Using TensorFlow backend.


In [2]:
NYU_CLASSES = ['bathtub', 'bed', 'bookshelf', 'box', 'chair', 'counter', 'desk', 'door', 'dresser',
               'garbage_bin', 'lamp', 'monitor', 'night_stand', 'pillow', 'sink', 'sofa', 'table', 'tv', 'toilet']
NUM_CLASSES = len(NYU_CLASSES) + 1
input_shape = (300, 300, 3) #channel lastde
depth_input_shape = (300, 300, 1)

In [3]:
ver = 7
np.random.seed(7)

In [4]:
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights('/media/hdd2/jun/checkpoints/bmvc/RGB/v{:d}/weights.best.hdf5'.format(ver), by_name=True)

In [5]:
rgb_gt = pickle.load(open('../pkls/RGB.pkl', 'rb'), encoding='latin1')
rgb_keys = sorted(rgb_gt.keys())
rgb_keys = np.array(rgb_keys)

perm = np.random.permutation(len(rgb_keys))
split = np.split(perm, 10)

test_perm = split.pop(ver)
ind = []
for ary in split:
    ind += list(ary)

num_train = int(len(ind) * 0.96)
rgb_train_keys = rgb_keys[ind[:num_train]]

rgb_val_keys = rgb_keys[ind[num_train:]]
rgb_test_keys = rgb_keys[test_perm]
num_train = len(rgb_train_keys)
num_val = len(rgb_val_keys)
num_test = len(rgb_test_keys)


In [6]:
path_prefix = '/media/hdd2/jun/dataset/'
rgb_inputs = []
depth_inputs = []
images = []
# img_path = path_prefix + sorted(val_keys )[100]
# img = image.load_img(img_path, target_size=(300, 300))
# img = image.img_to_array(img)
# images.append(imread(img_path))
# inputs.append(img.copy())
# inputs = preprocess_input(np.array(inputs))

for rgb_key in rgb_test_keys:
    rgb_img_path = path_prefix + rgb_key
    rgb_img = image.load_img(rgb_img_path, target_size=(300, 300))
    img = imread(rgb_img_path)
    images.append(img)
    rgb_img = image.img_to_array(rgb_img)
    rgb_inputs.append(rgb_img.copy())
    
inputs = preprocess_input(np.array(rgb_inputs))

In [7]:
priors = pickle.load(open('../pkls/prior_boxes_ssd300.pkl', 'rb'))
bbox_util = BBoxUtility(NUM_CLASSES, priors)

now = time.time()
preds = model.predict(inputs, batch_size=1, verbose=1)
finish = time.time()
duration = finish - now
print(duration)
print(966/duration)

966/966 [==============================] - 23s    
23.044214010238647
41.919416282577565


In [8]:
results = bbox_util.detection_out(preds,  confidence_threshold=0.5)

In [9]:
gt_bboxes = []
gt_labels = []
gt_scores = []
for key in rgb_test_keys:
    index = np.where(rgb_gt[key][:, 4:] == 1)
    gt_bboxes.append(rgb_gt[key][:, :4])
    gt_labels.append((index[1]).reshape(len(index[1]), 1))
    gt_scores.append(np.ones((len(index[1]), 1)))
gt_bboxes = np.array(gt_bboxes)
gt_labels = np.array(gt_labels)
gt_scores = np.array(gt_scores)

In [10]:
pred_labels = []
pred_scores = []
pred_bboxes = []
for result in results:
    if len(result) != 0:
        nm = len(result[:, 1])
        #pred_labels.append((result[:, 0]-1).reshape(nm, 1))
        pred_labels.append((result[:, 0]-1).reshape(nm, 1))
        pred_scores.append(result[:, 1:2].reshape(nm, 1))
        pred_bboxes.append(result[:, 2:].reshape(nm, 4))
    else:
        pred_labels.append(np.array([]).reshape(0, 1))
        pred_scores.append(np.array([]).reshape(0, 1))
        pred_bboxes.append(np.array([]).reshape(0, 1))
pred_labels = np.array(pred_labels)
pred_scores = np.array(pred_scores)

pred_bboxes = np.array(pred_bboxes)

In [11]:
prec, rec = calc_detection_prec_rec(pred_labels, pred_scores, pred_bboxes, gt_bboxes, gt_labels, iou_thresh=0.5)

In [12]:
ap = calc_detection_ap(prec, rec, use_07_metric=False)
{'ap': ap, 'map': np.nanmean(ap)}

{'ap': array([ 0.14166667,  0.61727984,  0.40270091,  0.01238992,  0.41529829,
         0.32532651,  0.1765829 ,  0.08597216,  0.24782009,  0.19379786,
         0.15197578,  0.21470953,  0.31810513,  0.36361246,  0.38027729,
         0.42019334,  0.39123896,  0.28518345,  0.75491158]),
 'map': 0.31047593092182679}

In [13]:
CLASSES = [{'id': 1, 'name': 'bathtub'}, {'id': 2, 'name': 'bed'}, {'id': 3, 'name': 'bookshelf'},
           {'id': 4, 'name': 'box'}, {'id': 5, 'name': 'chair'}, {'id': 6, 'name': 'counter'}, 
           {'id': 7, 'name': 'desk'}, {'id': 8, 'name': 'door'}, {'id': 9, 'name': 'dresser'}, 
           {'id': 10, 'name': 'garbage_bin'}, {'id': 11, 'name': 'lamp'}, {'id': 12, 'name': 'monitor'},
           {'id': 13, 'name': 'night_stand'}, {'id': 14, 'name': 'pillow'}, {'id': 15, 'name': 'sink'},
           {'id': 16, 'name': 'sofa'}, {'id': 17, 'name': 'table'},
           {'id': 18, 'name': 'tv'}, {'id': 19, 'name': 'toilet'}]
evaluator = ObjectDetectionEvaluator(CLASSES)

In [14]:
gt_bboxes = []
gt_labels = []
gt_scores = []
for key in rgb_test_keys:
    index = np.where(rgb_gt[key][:, 4:] == 1)
    gt_bboxes.append(rgb_gt[key][:, :4])
    gt_labels.append((index[1].tolist()))
    gt_scores.append(np.ones((len(index[1]), 1)))
gt_bboxes = np.array(gt_bboxes)
gt_labels = np.array(gt_labels)
gt_scores = np.array(gt_scores)

In [15]:
N = len(gt_labels)
for i in range(N):
    if len(pred_bboxes[i]) != 0:
        gt_dict = {standard_fields.InputDataFields.groundtruth_boxes: gt_bboxes[i], 
         standard_fields.InputDataFields.groundtruth_classes: np.array(gt_labels[i])
        }
        detected_dict = {
            standard_fields.DetectionResultFields.detection_boxes: pred_bboxes[i],
            standard_fields.DetectionResultFields.detection_classes: pred_labels[i].flatten(),
            standard_fields.DetectionResultFields.detection_scores: pred_scores[i].flatten()

        }
        evaluator.add_single_ground_truth_image_info(i, gt_dict)
        evaluator.add_single_detected_image_info(i, detected_dict)

In [16]:
metrics_dict = evaluator.evaluate()

In [21]:
metrics_dict[1]

{'PR_curve@bathtub': {'precisions': array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          0.97826087,  0.9787234 ,  0.97916667,  0.97959184,  0.98      ,
          0.98039216,  0.98076923,  0.98113208,  0.98148148,  0.98181818,
          0.98214286,  0.98245614,  0.98275862,  0.96610169,  0.95      ,
          0.95081967,  0.9516129 ,  0.95238095,  0.9375    ,  0.92307692,
    